In [ ]:
import pandas as pd
import numpy as np

import panel as pn
import holoviews as hv
from holoviews import opts
from holoviews import streams
from bokeh.models.formatters import PrintfTickFormatter
import seaborn as sns

# define a css class for a nice looking border of the plots
css = '''
.bk.plot-box {
  background: #ffffff;
  border-radius: 10px;
  border: 1px black solid;
}
'''

pn.extension(raw_css=[css])
hv.extension('bokeh', width=100)

# Loading the Data

## Sea level rise data

In [ ]:
df_slr_n = pd.read_csv('global_slr_netzero.csv', index_col=0)
df_slr_c = pd.read_csv('global_slr_zeroCO2.csv', index_col=0)

# Actual App

## Define possible values and styles

In [ ]:
cmap_years = sns.color_palette("flare", as_cmap=True)
peak_years_all_dict = {2020: cmap_years(0),
                       2025: cmap_years(1 / 7),
                       2030: cmap_years(2 / 7),
                       2035: cmap_years(3 / 7),
                       2040: cmap_years(4 / 7),
                       2045: cmap_years(5 / 7),
                       2050: cmap_years(6 / 7)}
cmap_dec = sns.color_palette("crest", as_cmap=True)
dec_rates_all_dict = {0.3: 'solid', 0.5: 'dashed', 0.7: 'dotted'}

peak_years_all = list(peak_years_all_dict)
dec_rates_all = list(dec_rates_all_dict)

## menu for selection

In [ ]:
# two selection options
select_year_slider = pn.widgets.IntSlider(start=2025, end=2301, step=5, value=2100, name='contribution at year')
select_slr_slider = pn.widgets.IntSlider(start=0, end=250, step=1, value=150, name='reach slr')
menu = pn.WidgetBox('Select year of contribution or SLR limit',
                    select_year_slider,
                    select_slr_slider)

## overview plot of all scenarios

In [ ]:
slr_plot_co2 = None
slr_plot_net = None

def get_single_slr_curve(time, slr, py, dec):
    return hv.Curve({'time': time,
                     'SLRG (mm)': slr,
                     'Peak Year': py,
                     'Decrease Rate': dec},
                    kdims=['time'],
                    vdims=['SLRG (mm)', 'Peak Year', 'Decrease Rate']
                   ).opts(tools=['hover'],
                          color=peak_years_all_dict[py],
                          line_dash=dec_rates_all_dict[dec],
                          responsive=True
                          )

for szenario in ['netzero', 'zeroCO2']:
    for py in peak_years_all:
        for decr in dec_rates_all:
            key = f'{szenario}_py{py}_fac1.0_decr{decr}'
            
            if szenario == 'netzero':
                slr_data = df_slr_n[key]
                
                if slr_plot_net is None:
                    slr_plot_net = get_single_slr_curve(slr_data.index.values,
                                                        slr_data.values,
                                                        py,
                                                        decr)
                else:
                    slr_plot_net *= get_single_slr_curve(slr_data.index.values,
                                                        slr_data.values,
                                                        py,
                                                        decr)
            elif szenario == 'zeroCO2':
                slr_data = df_slr_c[key]

                if slr_plot_co2 is None:
                    slr_plot_co2 = get_single_slr_curve(slr_data.index.values,
                                                        slr_data.values,
                                                        py,
                                                        decr)
                else:
                    slr_plot_co2 *= get_single_slr_curve(slr_data.index.values,
                                                        slr_data.values,
                                                        py,
                                                        decr)


## cross hair of main plots

In [ ]:
# dynamic plotting function for cross hair of selection
def draw_cross_hair(year, slr):
    return (hv.HLine(slr
                    ).opts(color='grey',
                           alpha=0.6,
                           line_width=1,
                           responsive=True) *
            hv.VLine(year
                    ).opts(color='grey',
                           alpha=0.5,
                           line_width=1,
                           responsive=True)
           )
dmap_cross_hair = hv.DynamicMap(
    pn.bind(draw_cross_hair,
            year=select_year_slider.param.value,
            slr=select_slr_slider.param.value)
)

## tap selection for main plots

In [ ]:
stream_click_selection_slr_co2 = hv.streams.Tap(x=2100, y=150, source=slr_plot_co2)
stream_click_selection_slr_net = hv.streams.Tap(x=2100, y=150, source=slr_plot_net)

def slr_tap_fct_c(x, y):
    select_year_slider.value = int(x)
    select_slr_slider.value = int(y)
    return None
def slr_tap_fct_n(x, y):
    select_year_slider.value = int(x)
    select_slr_slider.value = int(y)
    return None

slr_tap_co2 = pn.bind(slr_tap_fct_c,
                      x=stream_click_selection_slr_co2.param.x,
                      y=stream_click_selection_slr_co2.param.y)
slr_tap_net = pn.bind(slr_tap_fct_n,
                      x=stream_click_selection_slr_net.param.x,
                      y=stream_click_selection_slr_net.param.y)

## scatter plot glacier contribution to sea level rise in x

In [ ]:
def get_year_plot(year_selected, scenario):
    data = []
    if scenario == 'netzero':
        df_slr = df_slr_n
    elif scenario == 'zeroCO2':
        df_slr = df_slr_c
    for key, value in df_slr.loc[year_selected].items():
        s, yr, fac, de = key.split('_')
        yr = int(yr[2:])
        de = float(de[4:])
        data.append([yr, value, de])

    return hv.Points(data, kdims=['Year of peak emmisions', 'Sea level rise (mm)'], vdims=['dec']
                    ).opts(color='dec',
                           size=7,
                           title=f'Glacier contribution to sea level rise in {year_selected:.0f}',
                           ylim=(0, 250),
                           responsive=True,
                           cmap=cmap_dec,
                           tools=['hover'])

dmap_year_plot_c = hv.DynamicMap(
    pn.bind(get_year_plot,
            year_selected=select_year_slider.param.value,
            scenario='zeroCO2'))
dmap_year_plot_n = hv.DynamicMap(
    pn.bind(get_year_plot,
            year_selected=select_year_slider.param.value,
            scenario='netzero'))

## scatter plot year at which glacier contribution reaches x mm

In [ ]:
def get_slr_plot(slr_limit, scenario):
    data = []
    if scenario == 'netzero':
        df_slr = df_slr_n
    elif scenario == 'zeroCO2':
        df_slr = df_slr_c
    for key, value in df_slr.items():
        s, yr, fac, de = key.split('_')
        yr = int(yr[2:])
        de = float(de[4:])
        if np.all(value<slr_limit):
            year_reach = np.nan
        else:
            year_reach = (value<slr_limit).idxmin()
            
        data.append([yr, year_reach, de])

    return hv.Points(data, kdims=['Year of peak emmisions', 'Year'], vdims=['dec']
                    ).opts(color='dec',
                           size=7,
                           title=f'Year at which glacier contribution reaches {slr_limit:.0f} mm',
                           ylim=(2009, 2301),
                           responsive=True,
                           cmap=cmap_dec,
                           tools=['hover'])

dmap_slr_plot_c = hv.DynamicMap(
    pn.bind(get_slr_plot,
            slr_limit=select_slr_slider.param.value,
            scenario='zeroCO2'))
dmap_slr_plot_n = hv.DynamicMap(
    pn.bind(get_slr_plot,
            slr_limit=select_slr_slider.param.value,
            scenario='netzero'))

# Put the App together

In [ ]:
pn.Row(menu,
       pn.Spacer(width=10),
       pn.Column((slr_plot_co2 * dmap_cross_hair).opts(title='net-zero CO2'),
                 dmap_year_plot_c,
                 dmap_slr_plot_c,
                 css_classes=['plot-box'],
                 sizing_mode='stretch_both'),
       pn.Spacer(width=20),
       pn.Column((slr_plot_net * dmap_cross_hair).opts(title='net-zero GHG'),
                 dmap_year_plot_n,
                 dmap_slr_plot_n,
                 css_classes=['plot-box'],
                 sizing_mode='stretch_both'),
       slr_tap_co2,
       slr_tap_net,
       sizing_mode='stretch_both'
      ).servable()

Replace .servable() with .show() to show the app in a new browser tab.